In [143]:
import pandas as pd
import numpy as np
import docplex
from docplex.mp.model import Model
import matplotlib.pyplot as plt
import plotly.figure_factory as ff

## Instance 1

In [144]:
# Load the Excel file
file_path = "inst1.xlsx"
excel_data = pd.ExcelFile(file_path)

# Read data
unloading = excel_data.parse("Problem_1", index_col=None, header=0, usecols="B:E", nrows=5, skiprows=1)
cat = excel_data.parse("Problem_1", index_col=None, header=0, usecols="G:H", nrows=8, skiprows=1)
arrival = excel_data.parse("Problem_1", index_col=None, header=0, usecols="J:K", nrows=8, skiprows=1)

# indices
ships = np.arange(1, len(arrival)+1)
stations = np.arange(1, len(unloading.columns)+1)
categories = np.arange(1, len(unloading)+1)
names = {a+1: arrival.iloc[a,0] for a in range(0, len(arrival))}

# Changing input
unloading.index = categories
unloading.columns = stations
arrival = {a+1: arrival.iloc[a,1] for a in range(0, len(arrival))}

# BigM
BigM = 99999

# Combined parameters
conform = pd.DataFrame(index=categories, columns=stations)
for c in categories:
    for p in stations:
        conform.loc[c,p] = 1 if unloading.loc[c,p] < 1000 else 0

cat.iloc[:,0] = np.arange(1, len(cat)+1)
category = pd.DataFrame(0, index=ships, columns=categories)
for _ in range(0,len(cat)):
    category.loc[cat.iloc[_,0], cat.iloc[_,1]] = 1

In [147]:
def solve_model():
    # Creating the model
    model = Model(name="ship_sheduling", log_output=False)
    
    X = model.integer_var_matrix(ships, stations, name="Start")
    T = model.binary_var_matrix(ships, stations, name="Unloading")
    E = model.binary_var_matrix(ships, ships, name="Entering")
    C_max = model.integer_var(name="Makespan")

    # Constraints
    model.clear_constraints()

    # No early unlading
    model.add_constraints(X[s,1] >= arrival[s] for s in ships)

    # Ship must have passed previous station to reach new station
    for s in ships:
        for p in stations:
            for c in categories:
                if p > 1:
                    model.add_constraint(X[s,p] >= X[s,p-1] + unloading.loc[c,p-1] * category.loc[s,c] * T[s,p-1] + 1)

    # No overtaking and no two ships at one station
    for s1 in ships:
        for s2 in ships:
            if s1 != s2:
                for p in stations:
                    for c in categories:
                        model.add_constraint(X[s2,p] >= X[s1,p] + unloading.loc[c,p] * category.loc[s1,c] * T[s1,p] + (E[s1,s2] - 1) * BigM + 1)

    for s1 in ships:
        for s2 in ships:
            if s1 != s2:
                model.add_constraint(E[s1,s2] + E[s2,s1] == 1)

    # Ships must be unloaded once
    model.add_constraints(model.sum(T[s,p] for p in stations) == 1 for s in ships)

    # Ships can only be unloaded at conform stations
    model.add_constraints(T[s,p] * category.loc[s,c] <= conform.loc[c,p] for s in ships for p in stations for c in categories)

    # Setting the makespan
    model.add_constraints(C_max >= X[s,4] + unloading.loc[c,4] * category.loc[s,c] * T[s,4] for s in ships for c in categories)

    # Solving the model and printing the output
    model.minimize(C_max)
    model.solve()
    print(model.solve_details)
    print(f"The objective value is {model.objective_value} Periods")

    return model, X, T, E, C_max

In [3]:
# model = Model(name="ship_sheduling", log_output=False)

In [12]:
# model.clear()
# X = model.integer_var_matrix(ships, stations, name="Start")
# T = model.binary_var_matrix(ships, stations, name="Unloading")
# E = model.binary_var_matrix(ships, ships, name="Entering")
# C_max = model.integer_var(name="Makespan")

In [115]:
# model.clear_constraints()

# # No early unlading
# model.add_constraints(X[s,1] >= arrival[s] for s in ships)

# # Ship must have passed previous station to reach new station
# for s in ships:
#     for p in stations:
#         for c in categories:
#             if p > 1:
#                 model.add_constraint(X[s,p] >= X[s,p-1] + unloading.loc[c,p-1] * category.loc[s,c] * T[s,p-1] + 1)

# for s1 in ships:
#     for s2 in ships:
#         if s1 != s2:
#             for p in stations:
#                 for c in categories:
#                     model.add_constraint(X[s2,p] >= X[s1,p] + unloading.loc[c,p] * category.loc[s1,c] * T[s1,p] + (E[s1,s2] - 1) * BigM + 1)

# for s1 in ships:
#     for s2 in ships:
#         if s1 != s2:
#             model.add_constraint(E[s1,s2] + E[s2,s1] == 1)

# # Ships must be unloaded once
# model.add_constraints(model.sum(T[s,p] for p in stations) == 1 for s in ships)

# # Ships can only be unloaded at conform stations
# model.add_constraints(T[s,p] * category.loc[s,c] <= conform.loc[c,p] for s in ships for p in stations for c in categories)

# # Setting the makespan
# model.add_constraints(C_max >= X[s,4] + unloading.loc[c,4] * category.loc[s,c] * T[s,4] for s in ships for c in categories)

# print("Finish")

Finish


In [116]:
# model.minimize(C_max)
# model.solve()
# print(model.solve_details)
# print(f"The objective value is {model.objective_value} Periods")

status  = integer optimal solution
time    = 0.594 s.
problem = MILP
gap     = 0%

The objective value is 20.0 Periods


In [148]:
m, x, t, e, c_max = solve_model()

status  = integer optimal solution
time    = 0.812 s.
problem = MILP
gap     = 0%

The objective value is 20.0 Periods


#### Visualization

In [149]:
df = pd.DataFrame(index=np.arange(0, len(ships)*len(stations)), columns=["Task", "Start", "Finish", "Station"])
cnt = 0
for s in ships:
    for p in stations:
        for c in categories:
            if category.loc[s,c] == 1:
                ship_category = c
        df.iloc[cnt, 0] = names[s]
        df.iloc[cnt, 1] = (x[s,p].solution_value - 1) * 15
        if t[s,p].solution_value == 1:
            df.iloc[cnt, 2] = ((x[s,p].solution_value + unloading.loc[ship_category,p]) - 1) * 15
        else:
            df.iloc[cnt, 2] = x[s,p].solution_value * 15
        df.iloc[cnt,3] = p
        cnt += 1

colors = {1: "rgb(255, 0, 0)",
         2: "rgb(0, 255, 0)",
         3: "rgb(0, 0, 255)",
         4: "rgb(150, 150, 150)"}

fig = ff.create_gantt(df, colors=colors, index_col="Station", show_colorbar=True, group_tasks=True, bar_width=0.4)
fig.update_layout(xaxis_type='linear', autosize=False, width=800, height=600)
fig.show()

In [151]:
for s in ships:
    for p in stations:
        for c in categories:
            if category.loc[s,c] == 1:
                if t[s,p].solution_value == 1:
                    print(f"Ship {s} with category {c} is unloaded at jetty {p}")

Ship 1 with category 5 is unloaded at jetty 1
Ship 2 with category 1 is unloaded at jetty 1
Ship 3 with category 3 is unloaded at jetty 4
Ship 4 with category 4 is unloaded at jetty 4
Ship 5 with category 4 is unloaded at jetty 2
Ship 6 with category 2 is unloaded at jetty 3
Ship 7 with category 5 is unloaded at jetty 1
Ship 8 with category 5 is unloaded at jetty 1


## Instnace 2

In [158]:
# Load the Excel file
file_path = "inst2.xlsx"
excel_data = pd.ExcelFile(file_path)

# Read data
unloading = excel_data.parse("Problem_2", index_col=None, header=0, usecols="B:E", nrows=5, skiprows=1)
cat = excel_data.parse("Problem_2", index_col=None, header=0, usecols="G:H", nrows=10, skiprows=1)
arrival = excel_data.parse("Problem_2", index_col=None, header=0, usecols="J:K", nrows=10, skiprows=1)

# indices
ships = np.arange(1, len(arrival)+1)
stations = np.arange(1, len(unloading.columns)+1)
categories = np.arange(1, len(unloading)+1)
names = {a+1: arrival.iloc[a,0] for a in range(0, len(arrival))}

# Changing input
unloading.index = categories
unloading.columns = stations
arrival = {a+1: arrival.iloc[a,1] for a in range(0, len(arrival))}

# BigM
BigM = 99999

# Combined parameters
conform = pd.DataFrame(index=categories, columns=stations)
for c in categories:
    for p in stations:
        conform.loc[c,p] = 1 if unloading.loc[c,p] < 1000 else 0

cat.iloc[:,0] = np.arange(1, len(cat)+1)
category = pd.DataFrame(0, index=ships, columns=categories)
for _ in range(0,len(cat)):
    category.loc[cat.iloc[_,0], cat.iloc[_,1]] = 1

In [159]:
m.clear()
m, x, t, e, c_max = solve_model()

status  = integer optimal solution
time    = 3.344 s.
problem = MILP
gap     = 0%

The objective value is 24.0 Periods


#### Visualization

In [161]:
df = pd.DataFrame(index=np.arange(0, len(ships)*len(stations)), columns=["Task", "Start", "Finish", "Station"])
cnt = 0
for s in ships:
    for p in stations:
        for c in categories:
            if category.loc[s,c] == 1:
                ship_category = c
        df.iloc[cnt, 0] = names[s]
        df.iloc[cnt, 1] = (x[s,p].solution_value - 1) * 15
        if t[s,p].solution_value == 1:
            df.iloc[cnt, 2] = ((x[s,p].solution_value + unloading.loc[ship_category,p]) - 1) * 15
        else:
            df.iloc[cnt, 2] = x[s,p].solution_value * 15
        df.iloc[cnt,3] = p
        cnt += 1

colors = {1: "rgb(255, 0, 0)",
         2: "rgb(0, 255, 0)",
         3: "rgb(0, 0, 255)",
         4: "rgb(150, 150, 150)"}

fig = ff.create_gantt(df, colors=colors, index_col="Station", show_colorbar=True, group_tasks=True, bar_width=0.4)
fig.update_layout(xaxis_type='linear', autosize=False, width=800, height=600)
fig.show()

## Instance 3

In [163]:
# Load the Excel file
file_path = "inst3.xlsx"
excel_data = pd.ExcelFile(file_path)

# Read data
unloading = excel_data.parse("Problem_3", index_col=None, header=0, usecols="B:E", nrows=5, skiprows=1)
cat = excel_data.parse("Problem_3", index_col=None, header=0, usecols="G:H", nrows=8, skiprows=1)
arrival = excel_data.parse("Problem_3", index_col=None, header=0, usecols="J:K", nrows=8, skiprows=1)

# indices
ships = np.arange(1, len(arrival)+1)
stations = np.arange(1, len(unloading.columns)+1)
categories = np.arange(1, len(unloading)+1)
names = {a+1: arrival.iloc[a,0] for a in range(0, len(arrival))}

# Changing input
unloading.index = categories
unloading.columns = stations
arrival = {a+1: arrival.iloc[a,1] for a in range(0, len(arrival))}

# BigM
BigM = 99999

# Combined parameters
conform = pd.DataFrame(index=categories, columns=stations)
for c in categories:
    for p in stations:
        conform.loc[c,p] = 1 if unloading.loc[c,p] < 1000 else 0

cat.iloc[:,0] = np.arange(1, len(cat)+1)
category = pd.DataFrame(0, index=ships, columns=categories)
for _ in range(0,len(cat)):
    category.loc[cat.iloc[_,0], cat.iloc[_,1]] = 1

In [164]:
m.clear()
m, x, t, e, c_max = solve_model()

status  = integer optimal solution
time    = 0.656 s.
problem = MILP
gap     = 0%

The objective value is 20.0 Periods


#### Visualization

In [165]:
df = pd.DataFrame(index=np.arange(0, len(ships)*len(stations)), columns=["Task", "Start", "Finish", "Station"])
cnt = 0
for s in ships:
    for p in stations:
        for c in categories:
            if category.loc[s,c] == 1:
                ship_category = c
        df.iloc[cnt, 0] = names[s]
        df.iloc[cnt, 1] = (x[s,p].solution_value - 1) * 15
        if t[s,p].solution_value == 1:
            df.iloc[cnt, 2] = ((x[s,p].solution_value + unloading.loc[ship_category,p]) - 1) * 15
        else:
            df.iloc[cnt, 2] = x[s,p].solution_value * 15
        df.iloc[cnt,3] = p
        cnt += 1

colors = {1: "rgb(255, 0, 0)",
         2: "rgb(0, 255, 0)",
         3: "rgb(0, 0, 255)",
         4: "rgb(150, 150, 150)"}

fig = ff.create_gantt(df, colors=colors, index_col="Station", show_colorbar=True, group_tasks=True, bar_width=0.4)
fig.update_layout(xaxis_type='linear', autosize=False, width=800, height=600)
fig.show()